# Final Project Notebook: Comparing Neighborhoods in Windsor and Detroit

This project compares the cities Windsor and Detroit. This comparison is intresting to many as a lot of people choose to live in Windsor and comute to Detroit every day for work. This project tries to discover the differences between these cities and understand why people choses to live or work in one city verse the other.  

## Data:
The data used for this project will be scrapped using Beautiful Soup library from two wikipedia pages that hold the information about all the neighborhoods in Detroit and Windsor.

In [188]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from pandas import DataFrame

print('Libraries imported.')

Libraries imported.


### Web Scraping using Beautiful Soup library for the City Windsor

In [189]:
# import Beautiful Soup
from bs4 import BeautifulSoup
import requests
#get wikipedia page source html for windsor
source= requests.get('https://en.wikipedia.org/wiki/Neighbourhoods_of_Windsor,_Ontario').text
soup= BeautifulSoup(source, 'lxml')

In [190]:
#get the first borough
print(soup.h3.text)

Heart of Windsor[edit]


In [191]:
#get all the neighborhoods in windsor
span=soup.find_all('span', class_='mw-headline')
spanh2=soup.find_all('h2')
spanh2=spanh2[1:-4]

In [192]:
#create empty df with column neighborhood
df_windsor=DataFrame(columns=['Neighborhood'])
df_windsor

,Neighborhood


In [193]:
#get all boroughs and save in a df
df_borough=DataFrame(columns=['Borough'])
for sh2 in spanh2:
    sh2=sh2.text.replace('[edit]', '')
    df_borough=df_borough.append({'Borough':sh2}, ignore_index=True)
df_borough

,Borough
0,Downtown
1,East End
2,West Side
3,South Side


In [194]:
# get all the neighborhoods and save in a df
for s in span:
    s=s.text
    df_windsor=df_windsor.append({'Neighborhood':s}, ignore_index=True)
df_windsor=df_windsor[:-3]
df_windsor.head()

,Neighborhood
0,Downtown
1,Heart of Windsor
2,Little Italy
3,Uptown Ottawa Street
4,South Walkerville


In [195]:
#add the borough for each neighborhood
df_windsor.loc[1:7, 'Borough']='Downtown'
df_windsor.loc[9:18, 'Borough']='East End'
df_windsor.loc[20:27, 'Borough']='West Side'
df_windsor.loc[29:, 'Borough']='South Side'
#df_windsor

In [196]:
#remove rows with nan
df_windsor.dropna(inplace=True)
#reset index after drop operation 
df_windsor.reset_index(drop=True, inplace=True)
df_windsor.head()

,Neighborhood,Borough
0,Heart of Windsor,Downtown
1,Little Italy,Downtown
2,Uptown Ottawa Street,Downtown
3,South Walkerville,Downtown
4,Walkerville,Downtown


### Web Scraping using Beautiful Soup library for the City Detroit

In [197]:
#create empty df with cloumns Neighborhood and borough
df_detroit=DataFrame(columns=['Neighborhood', 'Borough'])
df_detroit

,Neighborhood,Borough


In [198]:
#get wikipedia page source html
source_detroit= requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Detroit').text
soup_detroit= BeautifulSoup(source_detroit, 'lxml')
#get the div tag where info is stored
div=soup_detroit.find('div', class_='navbox')
tr=div.find_all('tr')
tr=tr[1:] # remove first row of table that consists of the Title: Neighborhoods in Detroit
for t2 in tr:
    l2=t2.find_all('li')
    th=t2.find_all('th')
    for t in th:
        #print(t.text)
        #print()
        for l3 in l2: 
            #print(l3.text)
            df_detroit=df_detroit.append({'Borough':t.text, 'Neighborhood':l3.text}, ignore_index=True)
        #print()

In [199]:
df_detroit.head()

,Neighborhood,Borough
0,Bricktown,Downtown
1,Broadway Ave.,Downtown
2,Campus Martius,Downtown
3,Capital Park,Downtown
4,Downtown,Downtown


In [200]:
#remove dupplicate entries
df_detroit=df_detroit[df_detroit.Borough!='Upper']
df_detroit=df_detroit[df_detroit.Borough!='Central']
df_detroit=df_detroit[df_detroit.Borough!='Lower']
df_detroit=df_detroit[df_detroit.Borough!='Mixed-use']
df_detroit=df_detroit[df_detroit.Borough!='Residential']
df_detroit.reset_index(drop=True, inplace=True)
df_detroit.head()

,Neighborhood,Borough
0,Bricktown,Downtown
1,Broadway Ave.,Downtown
2,Campus Martius,Downtown
3,Capital Park,Downtown
4,Downtown,Downtown


In [201]:
#add city and country to each dfs
df_windsor['City']='Windsor'
df_detroit['City']='Detroit'
df_windsor['Country']='CA'
df_detroit['Country']='US'
df_detroit.head()

,Neighborhood,Borough,City,Country
0,Bricktown,Downtown,Detroit,US
1,Broadway Ave.,Downtown,Detroit,US
2,Campus Martius,Downtown,Detroit,US
3,Capital Park,Downtown,Detroit,US
4,Downtown,Downtown,Detroit,US


In [202]:
df_windsor.shape

(30, 4)

### Get the Latitude and Longtitude for neighborhoods in windsor 

In [203]:
#Get the coordinates for each neighborhood
geolocator = Nominatim(user_agent="downtown_explorer")
lat=[] #intialize empty list lat to store all latitude values
lng=[] #intialize empty list lng to store all longitude values
for neighborhood in df_windsor['Neighborhood']:
    try:
        location = geolocator.geocode(neighborhood+', Windsor, Ontario, CA') #get the coordinates
        lat.append(location.latitude) #get latitude and append to the list lat
        lng.append(location.longitude) #get longtitude and append to the list lng
    except:
        #remove any neighborhoods from the windsor df for which the cordinates can't be found
        df_windsor=df_windsor[df_windsor.Neighborhood!=neighborhood] 

#print(lat)
#print(lng)
    
#print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

In [204]:
df_windsor["Latitude"]=lat #assign list lat to the windsor df with new column Latitude 
df_windsor["Longitude"]=lng #assign list lng to the windsor df with new column Longtitude
df_windsor.head()

,Neighborhood,Borough,City,Country,Latitude,Longitude
3,South Walkerville,Downtown,Windsor,CA,42.295651,-82.996032
4,Walkerville,Downtown,Windsor,CA,42.307091,-82.994213
5,Central Windsor,Downtown,Windsor,CA,42.317099,-83.035343
8,East Windsor,East End,Windsor,CA,42.312449,-82.978921
9,Ford City,East End,Windsor,CA,42.310345,-82.948673


### Plot the neighborhoods in Windsor on a map

In [205]:
#Get the coordinates for Windsor
address = 'Windsor, CA'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Windsor are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Windsor are 42.317099, -83.0353434.


In [206]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_windsor['Latitude'], df_windsor['Longitude'], df_windsor['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Get the Latitude and Longtitude for neighborhoods in Detroit

In [207]:
#Get the coordinates for each neighborhood
lat=[] 
lng=[]
for neighborhood in df_detroit['Neighborhood']:
    try:
        location = geolocator.geocode(neighborhood+', Detroit, Michigan, US')
        lat.append(location.latitude)
        lng.append(location.longitude)
    except:
        df_detroit=df_detroit[df_detroit.Neighborhood!=neighborhood]

In [208]:
#print(len(lat))
#print(len(lng))
#print(df_detroit.shape)
df_detroit["Latitude"]=lat
df_detroit["Longitude"]=lng
df_detroit.head()

,Neighborhood,Borough,City,Country,Latitude,Longitude
0,Bricktown,Downtown,Detroit,US,42.333291,-83.041225
1,Broadway Ave.,Downtown,Detroit,US,42.335432,-83.048239
2,Campus Martius,Downtown,Detroit,US,42.332544,-83.047451
3,Capital Park,Downtown,Detroit,US,42.343040,-83.446643
4,Downtown,Downtown,Detroit,US,42.325961,-83.056758


### Plot the neighborhoods in Detroit on a map

In [209]:
address = 'Detroit, Michigan, US'

geolocator = Nominatim(user_agent="downtown_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Detroit are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Detroit are 42.3315509, -83.0466403.


In [210]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_detroit['Latitude'], df_detroit['Longitude'], df_detroit['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Combine the data for both cities into one dataframe and plot on a map 

In [211]:
df_cities=df_windsor.append(df_detroit, ignore_index=True)
df_cities.head()

,Neighborhood,Borough,City,Country,Latitude,Longitude
0,South Walkerville,Downtown,Windsor,CA,42.295651,-82.996032
1,Walkerville,Downtown,Windsor,CA,42.307091,-82.994213
2,Central Windsor,Downtown,Windsor,CA,42.317099,-83.035343
3,East Windsor,East End,Windsor,CA,42.312449,-82.978921
4,Ford City,East End,Windsor,CA,42.310345,-82.948673


In [212]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_cities['Latitude'], df_cities['Longitude'], df_cities['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Use Foursquare API to get the venues in each neighborhood

In [213]:
# @hidden_cell'
CLIENT_ID = 'AX5DG4HSWTT24GDZKERVGHQ3VAQGZY4B3DQ13WGNDVCGSG0Q' # your Foursquare ID
CLIENT_SECRET = '2LME0HZD0NZTYQ54YKTZLI2AAKWQHYUBZLHTOZIEYUVUCIEZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AX5DG4HSWTT24GDZKERVGHQ3VAQGZY4B3DQ13WGNDVCGSG0Q
CLIENT_SECRET:2LME0HZD0NZTYQ54YKTZLI2AAKWQHYUBZLHTOZIEYUVUCIEZ


In [214]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare AP
radius = 500 # define radius
#function for getting the venues for all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#calling the function to get all the venues near each neighborhood
venues = getNearbyVenues(names=df_cities['Neighborhood'],
                                   latitudes=df_cities['Latitude'],
                                   longitudes=df_cities['Longitude']
                                  )

South Walkerville
Walkerville
Central Windsor
East Windsor
Ford City
Forest Glade
Fontainebleau
Pillette Road Village
Riverside
Bridgeview
Brighton Beach
Morton Industrial Park
Ojibway
Sandwich
South Cameron Woodlot
West Windsor
Devonshire Heights
Remington Park
Roseland
South Windsor
Bricktown
Broadway Ave.
Campus Martius
Capital Park
Downtown
Financial District
Greektown
Griswold Street
West Jefferson Avenue
Park Avenue
Randolph Street Commercial
Washington Blvd.
Woodward Avenue
Art Center
Brush Park
Cass Corridor
Cass-Davenport
Cass Park
Medical Center
East Ferry Avenue
Jeffries
Midtown
Wayne State University
Woodbridge
Woodward East
Woodward Avenue
Atkinson Ave.
Boston-Edison
Henry Ford Hospital
New Center
Piquette Ave.
Woodward Avenue
Chaldean Town
Detroit Golf Club
Martin Park
Palmer Park
Palmer Woods
Sherwood Forest
University District
University of Detroit Mercy
Chandler Park
Conant Gardens
Fox Creek
Greenbriar
Grixdale
Hamtramck
Krainz Woods
Milwaukee Junction
Mohican Regent
M

In [ ]:
print(venues.shape)
venues.head()

(2960, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,South Walkerville,42.295651,-82.996032,Tim Hortons,42.297945,-82.990985,Coffee Shop
1,South Walkerville,42.295651,-82.996032,Tim Hortons,42.299960,-82.997270,Coffee Shop
2,South Walkerville,42.295651,-82.996032,Meth-od Labs Studio,42.293681,-82.991614,Music Venue
3,South Walkerville,42.295651,-82.996032,Pioneer Energy,42.298368,-82.991526,Gas Station
4,Walkerville,42.307091,-82.994213,The Salvation Army Thrift Store,42.305977,-82.996527,Thrift / Vintage Store


### Exploratory Data Analysis

In [ ]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Art Center,28,28,28,28,28,28
Atkinson Ave.,6,6,6,6,6,6
Atkinson Avenue,6,6,6,6,6,6
Aviation,4,4,4,4,4,4
Bagley,6,6,6,6,6,6
Barton-McFarland,4,4,4,4,4,4
Belmont,23,23,23,23,23,23
Boston-Edison,4,4,4,4,4,4
Boynton,2,2,2,2,2,2


In [ ]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 239 uniques categories.


### Prepare Dataframe for K-means Clustering

In [ ]:
#create a df suitable for clustering 
# one hot encoding
df_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.set_index('Neighborhood', inplace=True)
df_onehot.reset_index(inplace=True)

df_onehot.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Credit Union,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Arena,Home Service,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighthouse,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Romanian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,South Walkerville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,South Walkerville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
#check mean for each venue category in each neighborhood
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Credit Union,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Arena,Home Service,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighthouse,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Romanian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,Art Center,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.071429,0.035714,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.107143,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [ ]:
#Display top5 venue catogory in each neighborhood
num_top_venues = 5

for hood in df_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Art Center----
            venue  freq
0            Café  0.11
1     Pizza Place  0.07
2        Boutique  0.07
3  History Museum  0.07
4      Art Museum  0.07


----Atkinson Ave.----
                    venue  freq
0            Intersection  0.33
1       Mobile Phone Shop  0.17
2  Furniture / Home Store  0.17
3           Garden Center  0.17
4          Hardware Store  0.17


----Atkinson Avenue----
                    venue  freq
0            Intersection  0.33
1       Mobile Phone Shop  0.17
2  Furniture / Home Store  0.17
3           Garden Center  0.17
4          Hardware Store  0.17


----Aviation----
               venue  freq
0  Convenience Store  0.25
1        IT Services  0.25
2        Pizza Place  0.25
3             Lounge  0.25
4               Park  0.00


----Bagley----
           venue  freq
0           Food  0.33
1   Intersection  0.17
2   Soccer Field  0.17
3      BBQ Joint  0.17
4  Grocery Store  0.17


----Barton-McFarland----
                 venue  freq
0          

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Art Center,Café,History Museum,Diner,Boutique,Pizza Place,Art Museum,New American Restaurant,Movie Theater,Park,Sushi Restaurant
1,Atkinson Ave.,Intersection,Mobile Phone Shop,Garden Center,Furniture / Home Store,Hardware Store,Duty-free Shop,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
2,Atkinson Avenue,Intersection,Mobile Phone Shop,Garden Center,Furniture / Home Store,Hardware Store,Duty-free Shop,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
3,Aviation,Convenience Store,IT Services,Pizza Place,Lounge,Women's Store,Electronics Store,Dog Run,Donut Shop,Dry Cleaner,Duty-free Shop
4,Bagley,Food,Intersection,Soccer Field,Grocery Store,BBQ Joint,Credit Union,Doctor's Office,Fast Food Restaurant,Farmers Market,Convenience Store


### K-means Clustering

In [ ]:
# set number of clusters
kclusters = 7

df_grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 3, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df_cities

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,Neighborhood,Borough,City,Country,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,South Walkerville,Downtown,Windsor,CA,42.295651,-82.996032,0.0,Coffee Shop,Music Venue,Gas Station,Women's Store,Dive Bar,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store
1,Walkerville,Downtown,Windsor,CA,42.307091,-82.994213,0.0,Music Store,Rental Car Location,Bar,Thrift / Vintage Store,Women's Store,Dive Bar,Farmers Market,Factory,Event Space,Ethiopian Restaurant
2,Central Windsor,Downtown,Windsor,CA,42.317099,-83.035343,0.0,Bar,Lounge,Café,Pub,Nightclub,Coffee Shop,Pizza Place,Italian Restaurant,Indian Restaurant,Bank
3,East Windsor,East End,Windsor,CA,42.312449,-82.978921,0.0,Basketball Court,Women's Store,Food,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store
4,Ford City,East End,Windsor,CA,42.310345,-82.948673,0.0,Coffee Shop,Pizza Place,Breakfast Spot,American Restaurant,Health & Beauty Service,Beer Store,Sandwich Place,Dance Studio,Fast Food Restaurant,Greek Restaurant


In [ ]:
import math
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    if math.isnan(cluster)!=True:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster-1)],
            fill=True,
            fill_color=rainbow[int(cluster-1)],
            fill_opacity=0.7).add_to(map_clusters)
    
       
map_clusters

### Identifying Sutaible labels for each cluster

In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown,-82.996032,0.0,Coffee Shop,Music Venue,Gas Station,Women's Store,Dive Bar,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store
1,Downtown,-82.994213,0.0,Music Store,Rental Car Location,Bar,Thrift / Vintage Store,Women's Store,Dive Bar,Farmers Market,Factory,Event Space,Ethiopian Restaurant
2,Downtown,-83.035343,0.0,Bar,Lounge,Café,Pub,Nightclub,Coffee Shop,Pizza Place,Italian Restaurant,Indian Restaurant,Bank
3,East End,-82.978921,0.0,Basketball Court,Women's Store,Food,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store
4,East End,-82.948673,0.0,Coffee Shop,Pizza Place,Breakfast Spot,American Restaurant,Health & Beauty Service,Beer Store,Sandwich Place,Dance Studio,Fast Food Restaurant,Greek Restaurant
6,East End,-82.972931,0.0,Martial Arts Dojo,Department Store,Bank,Bar,Theme Park Ride / Attraction,Diner,Sandwich Place,American Restaurant,Fast Food Restaurant,Event Space
7,East End,-82.955028,0.0,Construction & Landscaping,Business Service,Fast Food Restaurant,Dance Studio,Flea Market,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store
8,East End,-82.943854,0.0,Pharmacy,Pizza Place,Bank,Gas Station,Coffee Shop,Discount Store,Bar,Department Store,Grocery Store,Dry Cleaner
9,West Side,-83.050122,0.0,Greek Restaurant,Diner,Italian Restaurant,Ice Cream Shop,Duty-free Shop,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
11,West Side,-83.095453,0.0,Boat or Ferry,Factory,Concert Hall,Flower Shop,Flea Market,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store


In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,West,-83.213247,1.0,Art Gallery,Women's Store,Dive Bar,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store
96,West,-83.213247,1.0,Art Gallery,Women's Store,Dive Bar,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store


In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,South Side,-83.004887,2.0,Yoga Studio,Furniture / Home Store,Food,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store
68,East,-82.977877,2.0,Furniture / Home Store,Women's Store,Food,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store


In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,New Center,-83.320578,3.0,Intersection,Mobile Phone Shop,Garden Center,Furniture / Home Store,Hardware Store,Duty-free Shop,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
50,New Center,-83.058525,3.0,Art Gallery,Intersection,Mediterranean Restaurant,Women's Store,Doctor's Office,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space
67,East,-83.060059,3.0,Intersection,Mediterranean Restaurant,Art Gallery,Women's Store,Doctor's Office,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space
75,East,-82.995974,3.0,Flea Market,Wine Bar,Intersection,Women's Store,Dive Bar,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant
94,West,-83.283006,3.0,Intersection,Food,Women's Store,Dive Bar,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant
124,Historic Districts,-83.320578,3.0,Intersection,Mobile Phone Shop,Garden Center,Furniture / Home Store,Hardware Store,Duty-free Shop,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
148,Historic Districts,-83.058525,3.0,Art Gallery,Intersection,Mediterranean Restaurant,Women's Store,Doctor's Office,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Event Space


In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,East End,-82.911933,4.0,Pizza Place,Women's Store,Distillery,Fast Food Restaurant,Farmers Market,Factory,Event Space,Ethiopian Restaurant,Electronics Store,Duty-free Shop
